In [0]:
!pip install findspark

You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.


In [0]:
!pip install sparkmeasure

You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.


In [0]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.sql import functions as F
from pyspark.sql.functions import *
from pyspark.sql.types import *
import json
import findspark

In [0]:
findspark.init()

In [0]:
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()
sc=spark.sparkContext

In [0]:
df = spark.read.option("header",True) \
     .csv("/FileStore/tables/houses.csv")
display(df)

size,nb_rooms,garden,orientation,price
116.300633073418,1,1,Sud,284413.3068710591
194.3028748040095,2,0,Nord,237354.82009708052
92.69496171573587,2,0,Ouest,225301.99878400558
116.47994613889072,2,0,Nord,214482.04415369357
138.25642489718405,2,1,Est,289134.2657668252
254.9083895514043,1,1,Nord,279516.8787203982
116.24576857935243,3,1,Sud,324392.254300987
110.88938812358977,2,0,Est,229287.98497631942
157.73240249812508,1,0,Nord,208024.89015503184
144.3021720928561,2,1,Sud,314046.69837688


In [0]:
df = df.repartition(2)
print(f"Le nombre de partition est :  {df.rdd.getNumPartitions()}")

Le nombre de partition est :  2


In [0]:
df.printSchema()

root
 |-- size: string (nullable = true)
 |-- nb_rooms: string (nullable = true)
 |-- garden: string (nullable = true)
 |-- orientation: string (nullable = true)
 |-- price: string (nullable = true)



In [0]:
df = df.withColumn("nb_rooms",col("nb_rooms").cast("int"))
df = df.withColumn("price",col("price").cast("float"))
df.printSchema()

root
 |-- size: string (nullable = true)
 |-- nb_rooms: integer (nullable = true)
 |-- garden: string (nullable = true)
 |-- orientation: string (nullable = true)
 |-- price: float (nullable = true)



In [0]:
display(df)

size,nb_rooms,garden,orientation,price
180.816778847453,3,0,Nord,254134.97
160.37259706550628,1,0,Ouest,229767.22
121.80324724745742,3,0,Est,253476.25
92.69496171573587,2,0,Ouest,225302.0
254.9083895514043,1,1,Nord,279516.88
130.798400466375,1,0,Ouest,220657.31
208.76463632877162,3,0,Nord,260569.38
116.300633073418,1,1,Sud,284413.3
35.68305835180962,3,1,Est,249104.17
34.318009898317726,1,1,Ouest,211397.69


In [0]:
df1 = df.select(['nb_rooms', 'price']).where((col("nb_rooms") >= 3) | (col("price") < 200000))
display(df1)

nb_rooms,price
3,306908.75
3,350864.44
3,281181.12
3,320174.25
3,343792.44
3,260569.38
3,253476.25
3,269197.5
3,254134.97
3,324392.25


In [0]:
df1.explain(True)

== Parsed Logical Plan ==
'Filter (('nb_rooms >= 3) OR ('price < 200000))
+- Project [nb_rooms#121, price#128]
   +- Project [size#90, nb_rooms#121, garden#92, orientation#93, cast(price#94 as float) AS price#128]
      +- Project [size#90, cast(nb_rooms#91 as int) AS nb_rooms#121, garden#92, orientation#93, price#94]
         +- Repartition 2, true
            +- Relation [size#90,nb_rooms#91,garden#92,orientation#93,price#94] csv

== Analyzed Logical Plan ==
nb_rooms: int, price: float
Filter ((nb_rooms#121 >= 3) OR (price#128 < cast(200000 as float)))
+- Project [nb_rooms#121, price#128]
   +- Project [size#90, nb_rooms#121, garden#92, orientation#93, cast(price#94 as float) AS price#128]
      +- Project [size#90, cast(nb_rooms#91 as int) AS nb_rooms#121, garden#92, orientation#93, price#94]
         +- Repartition 2, true
            +- Relation [size#90,nb_rooms#91,garden#92,orientation#93,price#94] csv

== Optimized Logical Plan ==
Project [cast(nb_rooms#91 as int) AS nb_rooms#1

In [0]:
df.createOrReplaceTempView("mytable");
df2 = sqlContext.sql('select nb_rooms, price from mytable where nb_rooms >=3 or price < 200000')
display(df2)

nb_rooms,price
3,306908.75
3,350864.44
3,281181.12
3,320174.25
3,343792.44
3,260569.38
3,253476.25
3,269197.5
3,254134.97
3,324392.25


In [0]:
df2.explain(True)

== Parsed Logical Plan ==
'Project ['nb_rooms, 'price]
+- 'Filter (('nb_rooms >= 3) OR ('price < 200000))
   +- 'UnresolvedRelation [mytable], [], false

== Analyzed Logical Plan ==
nb_rooms: int, price: float
Project [nb_rooms#121, price#128]
+- Filter ((nb_rooms#121 >= 3) OR (price#128 < cast(200000 as float)))
   +- SubqueryAlias mytable
      +- View (`mytable`, [size#90,nb_rooms#121,garden#92,orientation#93,price#128])
         +- Project [size#90, nb_rooms#121, garden#92, orientation#93, cast(price#94 as float) AS price#128]
            +- Project [size#90, cast(nb_rooms#91 as int) AS nb_rooms#121, garden#92, orientation#93, price#94]
               +- Repartition 2, true
                  +- Relation [size#90,nb_rooms#91,garden#92,orientation#93,price#94] csv

== Optimized Logical Plan ==
Project [cast(nb_rooms#91 as int) AS nb_rooms#121, cast(price#94 as float) AS price#128]
+- Repartition 2, true
   +- Project [nb_rooms#91, price#94]
      +- Filter ((cast(nb_rooms#91 as int) 

In [0]:
df3 = df.groupBy("nb_rooms").max("price")
display(df3)

nb_rooms,max(price)
1,300974.97
3,350864.44
2,324535.72


In [0]:
df3.explain(True)

== Parsed Logical Plan ==
'Aggregate ['nb_rooms], ['nb_rooms, max(price#128) AS max(price)#165]
+- Project [size#90, nb_rooms#121, garden#92, orientation#93, cast(price#94 as float) AS price#128]
   +- Project [size#90, cast(nb_rooms#91 as int) AS nb_rooms#121, garden#92, orientation#93, price#94]
      +- Repartition 2, true
         +- Relation [size#90,nb_rooms#91,garden#92,orientation#93,price#94] csv

== Analyzed Logical Plan ==
nb_rooms: int, max(price): float
Aggregate [nb_rooms#121], [nb_rooms#121, max(price#128) AS max(price)#165]
+- Project [size#90, nb_rooms#121, garden#92, orientation#93, cast(price#94 as float) AS price#128]
   +- Project [size#90, cast(nb_rooms#91 as int) AS nb_rooms#121, garden#92, orientation#93, price#94]
      +- Repartition 2, true
         +- Relation [size#90,nb_rooms#91,garden#92,orientation#93,price#94] csv

== Optimized Logical Plan ==
Aggregate [nb_rooms#121], [nb_rooms#121, max(price#128) AS max(price)#165]
+- Project [cast(nb_rooms#91 as int)

In [0]:
df4 = sqlContext.sql('SELECT nb_rooms, MAX(price) FROM mytable GROUP BY nb_rooms')
display(df4)

nb_rooms,max(price)
1,300974.97
3,350864.44
2,324535.72


In [0]:
df4.explain(True)

== Parsed Logical Plan ==
'Aggregate ['nb_rooms], ['nb_rooms, unresolvedalias('MAX('price), None)]
+- 'UnresolvedRelation [mytable], [], false

== Analyzed Logical Plan ==
nb_rooms: int, max(price): float
Aggregate [nb_rooms#121], [nb_rooms#121, max(price#128) AS max(price)#201]
+- SubqueryAlias mytable
   +- View (`mytable`, [size#90,nb_rooms#121,garden#92,orientation#93,price#128])
      +- Project [size#90, nb_rooms#121, garden#92, orientation#93, cast(price#94 as float) AS price#128]
         +- Project [size#90, cast(nb_rooms#91 as int) AS nb_rooms#121, garden#92, orientation#93, price#94]
            +- Repartition 2, true
               +- Relation [size#90,nb_rooms#91,garden#92,orientation#93,price#94] csv

== Optimized Logical Plan ==
Aggregate [nb_rooms#121], [nb_rooms#121, max(price#128) AS max(price)#201]
+- Project [cast(nb_rooms#91 as int) AS nb_rooms#121, cast(price#94 as float) AS price#128]
   +- Repartition 2, true
      +- Project [nb_rooms#91, price#94]
         +- 

In [0]:
from sparkmeasure import StageMetrics
stagemetrics = StageMetrics(spark)

In [0]:
# First example, measure metrics aggregated at stage level
stagemetrics.begin()

spark.sql('SELECT nb_rooms, MAX(price) FROM mytable GROUP BY nb_rooms').show()

stagemetrics.end()


+--------+----------+
|nb_rooms|max(price)|
+--------+----------+
|       1| 300974.97|
|       3| 350864.44|
|       2| 324535.72|
+--------+----------+



In [0]:
# This prints a report of aggregated metrics values

stagemetrics.print_report()


Scheduling mode = FAIR
Spark Context default degree of parallelism = 8

Aggregated Spark stage metrics:
numStages => 3
numTasks => 4
elapsedTime => 1354 (1 s)
stageDuration => 1046 (1 s)
executorRunTime => 1338 (1 s)
executorCpuTime => 437 (0.4 s)
executorDeserializeTime => 99 (99 ms)
executorDeserializeCpuTime => 49 (49 ms)
resultSerializationTime => 0 (0 ms)
jvmGCTime => 0 (0 ms)
shuffleFetchWaitTime => 0 (0 ms)
shuffleWriteTime => 86 (86 ms)
resultSize => 6240 (6.0 KB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 17563648
recordsRead => 40
bytesRead => 0 (0 Bytes)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 46
shuffleTotalBlocksFetched => 4
shuffleLocalBlocksFetched => 4
shuffleRemoteBlocksFetched => 0
shuffleTotalBytesRead => 1643 (1643 Bytes)
shuffleLocalBytesRead => 1643 (1643 Bytes)
shuffleRemoteBytesRead => 0 (0 Bytes)
shuffleRemoteBytesReadToDisk => 0 (0 Bytes)
shuffleBytesWritten => 1643 (1643 Bytes)
shuff

In [0]:
# The metrics are exported into a DataFrame and registered as a Temporary View

spark.sql("show tables").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
|        |  mytable|       true|
+--------+---------+-----------+



In [0]:
df_clean = spark.read.option("header",True) \
     .csv("/FileStore/tables/house_clean.csv")
display(df_clean)

nb_rooms,price
1,284413.3068710591
2,237354.82009708052
2,225301.99878400558
2,214482.04415369357
2,289134.2657668252
1,279516.8787203982
3,324392.254300987
2,229287.98497631942
1,208024.89015503184
2,314046.69837688


In [0]:
df_clean = df_clean.repartition(2)
print(f"Le nombre de partition est :  {df_clean.rdd.getNumPartitions()}")

Le nombre de partition est :  2


In [0]:
df_clean.createOrReplaceTempView("mytableclean");

In [0]:
stagemetrics.begin()

spark.sql('SELECT nb_rooms, MAX(price) FROM mytableclean GROUP BY nb_rooms').show()

stagemetrics.end()

+--------+-----------------+
|nb_rooms|       max(price)|
+--------+-----------------+
|       1|300974.9582511748|
|       2|324535.7239558707|
|       3|350864.4419628611|
+--------+-----------------+



In [0]:
stagemetrics.print_report()


Scheduling mode = FAIR
Spark Context default degree of parallelism = 8

Aggregated Spark stage metrics:
numStages => 3
numTasks => 4
elapsedTime => 504 (0.5 s)
stageDuration => 412 (0.4 s)
executorRunTime => 344 (0.3 s)
executorCpuTime => 103 (0.1 s)
executorDeserializeTime => 78 (78 ms)
executorDeserializeCpuTime => 37 (37 ms)
resultSerializationTime => 0 (0 ms)
jvmGCTime => 0 (0 ms)
shuffleFetchWaitTime => 0 (0 ms)
shuffleWriteTime => 90 (90 ms)
resultSize => 6230 (6.0 KB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 50528256
recordsRead => 40
bytesRead => 0 (0 Bytes)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 46
shuffleTotalBlocksFetched => 4
shuffleLocalBlocksFetched => 4
shuffleRemoteBlocksFetched => 0
shuffleTotalBytesRead => 1839 (1839 Bytes)
shuffleLocalBytesRead => 1839 (1839 Bytes)
shuffleRemoteBytesRead => 0 (0 Bytes)
shuffleRemoteBytesReadToDisk => 0 (0 Bytes)
shuffleBytesWritten => 1839 (1839 Bytes)
sh